# **Exploratory Data Analysis**: FlipTop

## Question
----

The central question of this project revolves around FlipTop battle rap careers and their length.

> How long does a career in the Philippines' premier rap battle league FlipTop last?

Who rises to the top? What determines who stays relevant over the years?

Here I'm thinking survival (time-to-event) analysis.

Another question I'm curious about is this:
> Who has battled who? And what matchups haven't been done yet?

I'm picturing a graph here. I want to understand how dense or sparse the network of rap battles are in FlipTop.

Ideas:
- Emcees are nodes
- "Battled" as edges
- Weight of the edge is how many times they've battled?

## Data Wrangling
----

Data wrangled using [YouTube API](https://developers.google.com/youtube/v3).

In [1]:
import json
import pandas as pd
import numpy as np

Note: This data was scraped July 2, 2025.

In [2]:
# Load JSON data
with open("../data/videos.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Convert to DataFrame
df = pd.DataFrame(data)

# Preview
df.head()

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags
0,NCElL6MbZ_g,FlipTop - Caspher vs CRhyme,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-07-02T13:23:54Z,139884,PT32M32S,https://www.youtube.com/watch?v=NCElL6MbZ_g,3117,613,"[fliptop, flip top, flip top battles, fliptopb..."
1,JabvhPBmoVs,FlipTop CripLi - Isabuhay 2025 | Abangan si Cr...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:49:25Z,27498,PT1M51S,https://www.youtube.com/watch?v=JabvhPBmoVs,640,31,"[fliptop, flip top, flip top battles, fliptopb..."
2,YiJI_ohq4Pc,FlipTop Ban - Isabuhay 2025 | Abangan si Ban n...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:49:25Z,29353,PT1M24S,https://www.youtube.com/watch?v=YiJI_ohq4Pc,763,34,"[fliptop, flip top, flip top battles, fliptopb..."
3,yRE3PU0ekaA,FlipTop J-Blaque - Gods Pa Rin? | | Abangan si...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:49:25Z,17415,PT1M56S,https://www.youtube.com/watch?v=yRE3PU0ekaA,360,8,"[fliptop, flip top, flip top battles, fliptopb..."
4,Ftk2DZ3hcxw,FlipTop Vitrum - Ang HipHop Pinalakas ng mga T...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:48:58Z,28969,PT3M15S,https://www.youtube.com/watch?v=Ftk2DZ3hcxw,449,50,"[fliptop, flip top, flip top battles, fliptopb..."


In [ ]:
import sys
sys.path.append("../scripts")

from emcee_scraper import scrape_names
emcees = scrape_names()

Scraped 20 names from page 1.
Scraped 20 names from page 2.
Scraped 20 names from page 3.
Scraped 20 names from page 4.
Scraped 20 names from page 5.
Scraped 20 names from page 6.
Scraped 20 names from page 7.
Scraped 20 names from page 8.
Scraped 7 names from page 9.
No emcees found on page 10. Stopping early.


In [5]:
import os
import csv

out_path = os.path.join("..", "data", "emcees.csv")

with open(out_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["name"])
    for name in emcees:
        writer.writerow([name])

In [5]:
len(emcees)

167

## Data Cleaning
----

Tasks:

- convert `upload_date` to something more usable.
- convert `duration` to human readable (HH:MM:SS maybe?)
- Some shorts/round highlight moments were uploaded on the [videos playlist](https://www.youtube.com/@fliptopbattles/videos) of the channel so need to filter by duration, especially post-2020 (when YouTube Shorts became a thing).
- Only considering 1v1 battles that were judged (for now)
    - So remove:
        - Sound Check
        - Freestyle battles
        - Dos Por Dos (2v2)
        - Royal Rumble (1v1v1v1v1)
        - Those two 5v5 battles that happened only twice
        - Anygma Machine
        - Video Flyer
        - Announcement videos
        - Emcee interview videos

BIG TASKS

- Clean up `title` and create new column `battle_card` or something that contains only names e.g. Emcee1 vs Emcee2
- Webscrape [emcee page](https://www.fliptop.com.ph/emcees) to get a full and complete list of the emcees in FlipTop (according to their own website roster).
    - Good for later user of cross-referencing titles with emcee names.
- 

In [6]:
df.head()

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags
0,NCElL6MbZ_g,FlipTop - Caspher vs CRhyme,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-07-02T13:23:54Z,139884,PT32M32S,https://www.youtube.com/watch?v=NCElL6MbZ_g,3117,613,"[fliptop, flip top, flip top battles, fliptopb..."
1,JabvhPBmoVs,FlipTop CripLi - Isabuhay 2025 | Abangan si Cr...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:49:25Z,27498,PT1M51S,https://www.youtube.com/watch?v=JabvhPBmoVs,640,31,"[fliptop, flip top, flip top battles, fliptopb..."
2,YiJI_ohq4Pc,FlipTop Ban - Isabuhay 2025 | Abangan si Ban n...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:49:25Z,29353,PT1M24S,https://www.youtube.com/watch?v=YiJI_ohq4Pc,763,34,"[fliptop, flip top, flip top battles, fliptopb..."
3,yRE3PU0ekaA,FlipTop J-Blaque - Gods Pa Rin? | | Abangan si...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:49:25Z,17415,PT1M56S,https://www.youtube.com/watch?v=yRE3PU0ekaA,360,8,"[fliptop, flip top, flip top battles, fliptopb..."
4,Ftk2DZ3hcxw,FlipTop Vitrum - Ang HipHop Pinalakas ng mga T...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:48:58Z,28969,PT3M15S,https://www.youtube.com/watch?v=Ftk2DZ3hcxw,449,50,"[fliptop, flip top, flip top battles, fliptopb..."


In [8]:
# looking at desc
samp = df.sample(n=5, replace=False)
samp

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags
1105,NYHI3wJzeE0,FlipTop - Aelekz vs J-Blaque,FlipTop Visayas presents: Gubat 5 @ CAP Center...,2016-11-27T12:00:58Z,273669,PT24M19S,https://www.youtube.com/watch?v=NYHI3wJzeE0,621,295,"[fliptop, fliptop new, fliptop latest, fliptop..."
1463,gO8B75deIT8,FlipTop - Rapido vs Mocks Wun,"FlipTop Metro Manila presents: Ahon 4, Day 2 +...",2013-02-24T07:37:32Z,698182,PT20M21S,https://www.youtube.com/watch?v=gO8B75deIT8,1342,1579,"[rapido, mocks wun, fliptop, ahon 4, day 2, ba..."
714,MLBMHkQFwL8,FlipTop - LilWeng vs YoungOne,FlipTop Metro Manila presents: Ahon 10 @ TIU T...,2020-04-03T10:51:59Z,857280,PT23M16S,https://www.youtube.com/watch?v=MLBMHkQFwL8,12191,2248,"[fliptop, flip top, flip top battles, fliptopb..."
1095,Akf72KvdnHw,FlipTop - Melchrist vs Pistolero,FlipTop Metro Manila presents: Unibersikulo 4 ...,2016-12-31T12:18:53Z,469303,PT20M15S,https://www.youtube.com/watch?v=Akf72KvdnHw,1166,855,"[fliptop, melchrist, pistolero, fliptop new, f..."
123,6gOjF282aNg,FlipTop - Negho G / Pamoso vs Atoms / Cygnus @...,FlipTop presents: Gubat 14 @ IEC Convention Ce...,2024-12-18T14:09:05Z,482866,PT48M37S,https://www.youtube.com/watch?v=6gOjF282aNg,7635,1330,"[fliptop, flip top, flip top battles, fliptopb..."


In [12]:
samp['description']

1105    FlipTop Visayas presents: Gubat 5 @ CAP Center...
1463    FlipTop Metro Manila presents: Ahon 4, Day 2 +...
714     FlipTop Metro Manila presents: Ahon 10 @ TIU T...
1095    FlipTop Metro Manila presents: Unibersikulo 4 ...
123     FlipTop presents: Gubat 14 @ IEC Convention Ce...
Name: description, dtype: object

In [17]:
desc_test = samp['description'].loc[1105]
print(desc_test)

FlipTop Visayas presents: Gubat 5 @ CAP Center, Osmena Blvd., Metro Cebu Philippines. September 24, 2016. Filipino Conference Battle.

-AELEKZ VS J-BLAQUE-

Subscribe Here! http://bit.ly/fliptopsub
Check out our top videos! http://bit.ly/fliptopTopVideos

Website: https://fliptop.com.phFacebook: https://www.facebook.com/fliptop.battleleague
Twitter: https://twitter.com/FlipTop_Battles

 #fliptopbattles

About fliptopbattles:
FlipTop Kru Corp. is a self-produced events and artist management company with its first product in the FlipTop Battle League. The FlipTop Battle League is the Philippines’ first premier – and the world’s most-viewed – rap battle league. It is popularly credited for the resurgence and widespread acceptance of hiphop culture in the Philippines since its inception in February 2010, and continues to champion all other hiphop elements in its variety of events and online content. It is home to the country’s top Hiphop talents and houses divisions in the main island grou

Video descriptions often have good info on them for when and where the rap battle took place. Need to scrape these too!